In [1]:
import pandas as pd
import re

In [12]:
# Load Excel file and retrieve sheet names
xlsx_file = 'data\\raw\\online_retail_II.xlsx' 

sheet_names = pd.ExcelFile(xlsx_file).sheet_names
print(sheet_names)

['Year 2009-2010', 'Year 2010-2011']


In [14]:
# Load data from specific sheets in the Excel file
df1 = pd.read_excel(xlsx_file, sheet_name='Year 2009-2010')
df2 = pd.read_excel(xlsx_file, sheet_name='Year 2010-2011')

In [16]:
# Concatinate the two sheets
df = pd.concat([df1, df2], ignore_index=True)
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [18]:
#dataset overview

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


In [22]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,1.067371e+06,1067371,1.067371e+06,824364.000000
mean,9.938898e+00,2011-01-02 21:13:55.394028544,4.649388e+00,15324.638504
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-09 09:46:00,1.250000e+00,13975.000000
50%,3.000000e+00,2010-12-07 15:28:00,2.100000e+00,15255.000000
75%,1.000000e+01,2011-07-22 10:23:00,4.150000e+00,16797.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.727058e+02,NaN,1.235531e+02,1697.464450


In [24]:
#Dataset cleaning and separation

In [26]:
# Replace missing customer IDs with 'Unknown' and convert to string type
df['Customer ID'] = df['Customer ID'].fillna('Unknown').astype(str)

In [28]:
# Clean customer IDs: retain 'Unknown' or convert numeric strings to integers as strings
def clean_customer_id(value):
    if value == 'Unknown':  # Keep 'unknown' unchanged
        return value
    else:
        return str(int(float(value)))  # Convert valid numeric strings
df['Customer ID'] = df['Customer ID'].apply(clean_customer_id)

In [30]:
# Fill missing descriptions using a lookup dictionary based on 'StockCode'
def fill_missing_desc(row, lookup):
    if pd.isna(row['Description']):
        matching_desc = lookup.get(row['StockCode'], 'Unknown')
        return matching_desc
    return row['Description']
lookup_dict = df[df['Description'].notna()].set_index('StockCode')['Description'].to_dict()

df['Description'] = df.apply(lambda row: fill_missing_desc(row, lookup_dict), axis=1)


In [32]:
#Convert to date time
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [34]:
# Separate the cancellations from the regular transactions
def sep_cans(df):
    cancellations_mask = df['Invoice'].str.startswith('C', na=False)
    cancellations = df[cancellations_mask]
    sales = df[~cancellations_mask]
    
    return sales, cancellations


s_df, c_df = sep_cans(df)

In [36]:
#sales dataset further cleaning and duplication handling

In [38]:
s_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1047877 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1047877 non-null  object        
 1   StockCode    1047877 non-null  object        
 2   Description  1047877 non-null  object        
 3   Quantity     1047877 non-null  int64         
 4   InvoiceDate  1047877 non-null  datetime64[ns]
 5   Price        1047877 non-null  float64       
 6   Customer ID  1047877 non-null  object        
 7   Country      1047877 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 72.0+ MB


In [40]:
# Filter invoices starting with 'A' that are not transactional
adjst_bd_dbt_mask = s_df['Invoice'].str.startswith('A', na=False)
adjst_bd_dbt = s_df[adjst_bd_dbt_mask]
adjst_bd_dbt


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,Unknown,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,Unknown,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,Unknown,United Kingdom
825443,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,Unknown,United Kingdom
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,Unknown,United Kingdom
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,Unknown,United Kingdom


In [42]:
s_df = s_df[~adjst_bd_dbt_mask]
s_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France


In [44]:
invld_qte = s_df[(s_df['Quantity'] <= 0)  & (s_df['Price'] <= 0)]
invld_qte

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.0,Unknown,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.0,Unknown,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.0,Unknown,United Kingdom
470,489521,21646,Unknown,-50,2009-12-01 11:44:00,0.0,Unknown,United Kingdom
3114,489655,20683,RAIN GIRL CHILDS UMBRELLA,-44,2009-12-01 17:26:00,0.0,Unknown,United Kingdom
...,...,...,...,...,...,...,...,...
1060794,581210,23395,check,-26,2011-12-07 18:36:00,0.0,Unknown,United Kingdom
1060796,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,Unknown,United Kingdom
1060797,581213,22576,check,-30,2011-12-07 18:38:00,0.0,Unknown,United Kingdom
1062371,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,Unknown,United Kingdom


In [46]:
# Drop heaviley incomplete data
s_df = s_df.drop(invld_qte.index)

In [50]:
# Fill missing prices with values from the lookup dictionary where StockCode matches
def fill_missing_price(row, lookup):
    if row['Price'] == 0:
        matching_price = lookup.get(row['StockCode'], 0.0)
        return matching_price
    return row['Price']
lookup_dict = s_df[s_df['Price'].ne(0)].set_index('StockCode')['Price'].to_dict()

s_df['Price'] = s_df.apply(lambda row: fill_missing_price(row, lookup_dict), axis=1)

In [52]:
# Drop the invalid data
invld_prc_zero = s_df[s_df['Price'] == 0]
invld_prc_zero

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
43528,493138,35751A,Unknown,152,2009-12-22 11:32:00,0.0,Unknown,United Kingdom
43583,493150,71263G,Unknown,39,2009-12-22 11:38:00,0.0,Unknown,United Kingdom
43623,493162,20691,Unknown,20,2009-12-22 11:49:00,0.0,Unknown,United Kingdom
43625,493164,20880,Unknown,17,2009-12-22 11:49:00,0.0,Unknown,United Kingdom
43626,493165,20859,Unknown,15,2009-12-22 11:50:00,0.0,Unknown,United Kingdom
...,...,...,...,...,...,...,...,...
366735,524956,49031B,CHROME EURO HOOK 20cm,400,2010-10-01 13:23:00,0.0,Unknown,United Kingdom
779591,559296,23001,TRAVEL CARD WALLET DOTCOMGIFTSHOP,200,2011-07-07 12:35:00,0.0,Unknown,United Kingdom
872310,567207,35592T,Unknown,4,2011-09-19 11:01:00,0.0,Unknown,United Kingdom
942568,572614,23595,adjustment,5,2011-10-25 11:38:00,0.0,Unknown,United Kingdom


In [54]:
invld_prc_zero = invld_prc_zero.drop([43639, 43837, 43853, 366735, 779591])
invld_prc_zero

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
43528,493138,35751A,Unknown,152,2009-12-22 11:32:00,0.0,Unknown,United Kingdom
43583,493150,71263G,Unknown,39,2009-12-22 11:38:00,0.0,Unknown,United Kingdom
43623,493162,20691,Unknown,20,2009-12-22 11:49:00,0.0,Unknown,United Kingdom
43625,493164,20880,Unknown,17,2009-12-22 11:49:00,0.0,Unknown,United Kingdom
43626,493165,20859,Unknown,15,2009-12-22 11:50:00,0.0,Unknown,United Kingdom
...,...,...,...,...,...,...,...,...
298839,518487,gift_0001_90,Unknown,10,2010-08-09 12:28:00,0.0,Unknown,United Kingdom
298843,518490,gift_0001_60,Unknown,10,2010-08-09 12:29:00,0.0,Unknown,United Kingdom
872310,567207,35592T,Unknown,4,2011-09-19 11:01:00,0.0,Unknown,United Kingdom
942568,572614,23595,adjustment,5,2011-10-25 11:38:00,0.0,Unknown,United Kingdom


In [56]:
s_df = s_df.drop(invld_prc_zero.index)

In [58]:
s_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1044349 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1044349 non-null  object        
 1   StockCode    1044349 non-null  object        
 2   Description  1044349 non-null  object        
 3   Quantity     1044349 non-null  int64         
 4   InvoiceDate  1044349 non-null  datetime64[ns]
 5   Price        1044349 non-null  float64       
 6   Customer ID  1044349 non-null  object        
 7   Country      1044349 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 71.7+ MB


In [60]:
s_df['Invoice'] = s_df['Invoice'].astype(str)
s_df['StockCode'] = s_df['StockCode'].astype(str)

In [62]:
s_df['Invoice'] = s_df['Invoice'].str.strip()
s_df['StockCode'] = s_df['StockCode'].str.strip()

In [64]:
# Add a new column 'RowIndex' to the dataframe with the index values to handle duplicates
s_df['RowIndex'] = s_df.index
s_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,3
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,4
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,1067366
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,1067367
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,1067368
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,1067369


In [66]:
# Identify and sort duplicated rows based on 'Invoice' and 'StockCode' columns
dup = s_df[s_df.duplicated(subset=['Invoice', 'StockCode'], keep=False)]

dup_sort = dup.sort_values(by=['Invoice', 'StockCode'])
dup_sort

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
289,489488,22125,UNION JACK HOT WATER BOTTLE,2,2009-12-01 10:59:00,5.95,17238,United Kingdom,289
299,489488,22125,UNION JACK HOT WATER BOTTLE,1,2009-12-01 10:59:00,5.95,17238,United Kingdom,299
379,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329,United Kingdom,379
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329,United Kingdom,391
373,489517,21790,VINTAGE SNAP CARDS,4,2009-12-01 11:34:00,0.85,16329,United Kingdom,373
...,...,...,...,...,...,...,...,...,...
1067141,581538,23349,ROLL WRAP VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,1.25,14446,United Kingdom,1067141
1067135,581538,35004B,SET OF 3 BLACK FLYING DUCKS,2,2011-12-09 11:34:00,5.45,14446,United Kingdom,1067135
1067159,581538,35004B,SET OF 3 BLACK FLYING DUCKS,1,2011-12-09 11:34:00,5.45,14446,United Kingdom,1067159
1067103,581538,84380,SET OF 3 BUTTERFLY COOKIE CUTTERS,1,2011-12-09 11:34:00,1.45,14446,United Kingdom,1067103


In [68]:
# Remove duplicate rows based on 'Invoice', 'StockCode', 'Quantity', and 'InvoiceDate', keeping the first occurrence
dup_sort = dup_sort.drop_duplicates(subset=['Invoice', 'StockCode','Quantity', 'InvoiceDate'], keep='first')
dup_sort

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
289,489488,22125,UNION JACK HOT WATER BOTTLE,2,2009-12-01 10:59:00,5.95,17238,United Kingdom,289
299,489488,22125,UNION JACK HOT WATER BOTTLE,1,2009-12-01 10:59:00,5.95,17238,United Kingdom,299
379,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329,United Kingdom,379
373,489517,21790,VINTAGE SNAP CARDS,4,2009-12-01 11:34:00,0.85,16329,United Kingdom,373
393,489517,21790,VINTAGE SNAP CARDS,1,2009-12-01 11:34:00,0.85,16329,United Kingdom,393
...,...,...,...,...,...,...,...,...,...
1067141,581538,23349,ROLL WRAP VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,1.25,14446,United Kingdom,1067141
1067135,581538,35004B,SET OF 3 BLACK FLYING DUCKS,2,2011-12-09 11:34:00,5.45,14446,United Kingdom,1067135
1067159,581538,35004B,SET OF 3 BLACK FLYING DUCKS,1,2011-12-09 11:34:00,5.45,14446,United Kingdom,1067159
1067103,581538,84380,SET OF 3 BUTTERFLY COOKIE CUTTERS,1,2011-12-09 11:34:00,1.45,14446,United Kingdom,1067103


In [78]:
# Keep the row with the maximum 'Quantity' for each duplicate group
dup_sort = (
    dup_sort.groupby(['Invoice', 'StockCode'], as_index=False) 
       .apply(lambda group: group.loc[group['Quantity'].idxmax()])  
       .reset_index(drop=True)
)
dup_sort

C:\Users\user\AppData\Local\Temp\ipykernel_2832\2514455002.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.loc[group['Quantity'].idxmax()])


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
0,489488,22125,UNION JACK HOT WATER BOTTLE,2,2009-12-01 10:59:00,5.95,17238,United Kingdom,289
1,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329,United Kingdom,379
2,489517,21790,VINTAGE SNAP CARDS,4,2009-12-01 11:34:00,0.85,16329,United Kingdom,373
3,489517,21791,VINTAGE HEADS AND TAILS CARD GAME,6,2009-12-01 11:34:00,1.25,16329,United Kingdom,372
4,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329,United Kingdom,365
...,...,...,...,...,...,...,...,...,...
42079,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446,United Kingdom,1067117
42080,581538,23343,JUMBO BAG VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,2.08,14446,United Kingdom,1067127
42081,581538,23349,ROLL WRAP VINTAGE CHRISTMAS,3,2011-12-09 11:34:00,1.25,14446,United Kingdom,1067111
42082,581538,35004B,SET OF 3 BLACK FLYING DUCKS,2,2011-12-09 11:34:00,5.45,14446,United Kingdom,1067135


In [80]:
dup_check = dup_sort[dup_sort.duplicated(subset=['Invoice', 'StockCode'], keep=False)]
dup_check

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex


In [84]:
dup_rows = dup[~dup['RowIndex'].isin(dup_sort['RowIndex'])]
dup_rows

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
299,489488,22125,UNION JACK HOT WATER BOTTLE,1,2009-12-01 10:59:00,5.95,17238,United Kingdom,299
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329,United Kingdom,371
383,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329,United Kingdom,383
384,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,16329,United Kingdom,384
385,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,16329,United Kingdom,385
...,...,...,...,...,...,...,...,...,...
1067153,581538,22992,REVOLVER WOODEN RULER,1,2011-12-09 11:34:00,1.95,14446,United Kingdom,1067153
1067158,581538,21194,PINK HONEYCOMB PAPER FAN,1,2011-12-09 11:34:00,0.65,14446,United Kingdom,1067158
1067159,581538,35004B,SET OF 3 BLACK FLYING DUCKS,1,2011-12-09 11:34:00,5.45,14446,United Kingdom,1067159
1067160,581538,22694,WICKER STAR,1,2011-12-09 11:34:00,2.10,14446,United Kingdom,1067160


In [86]:
sc_df = s_df[~s_df['RowIndex'].isin(dup_rows['RowIndex'])]
sc_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,3
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,4
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,1067366
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,1067367
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,1067368
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,1067369


In [88]:
sc_df = sc_df.drop(columns=['RowIndex'])

In [90]:
#Checking consistancy

In [92]:
sc_df['Country'].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Denmark', 'Netherlands', 'Poland',
       'Channel Islands', 'Spain', 'Cyprus', 'Greece', 'Norway',
       'Austria', 'Sweden', 'United Arab Emirates', 'Finland', 'Italy',
       'Switzerland', 'Japan', 'Unspecified', 'Nigeria', 'Malta',
       'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore', 'Thailand',
       'Israel', 'Lithuania', 'West Indies', 'Lebanon', 'Korea', 'Brazil',
       'Canada', 'Iceland', 'Saudi Arabia', 'Czech Republic',
       'European Community'], dtype=object)

In [94]:
# Replace country abbreviations with full names
country_mapping = {
    'EIRE': 'Ireland',
    'RSA': 'South Africa',
    'USA': 'United States',
    'Korea': 'South Korea'  
}
sc_df['Country'] = sc_df['Country'].replace(country_mapping)
sc_df['Country'].unique()

array(['United Kingdom', 'France', 'United States', 'Belgium',
       'Australia', 'Ireland', 'Germany', 'Portugal', 'Denmark',
       'Netherlands', 'Poland', 'Channel Islands', 'Spain', 'Cyprus',
       'Greece', 'Norway', 'Austria', 'Sweden', 'United Arab Emirates',
       'Finland', 'Italy', 'Switzerland', 'Japan', 'Unspecified',
       'Nigeria', 'Malta', 'Bahrain', 'South Africa', 'Bermuda',
       'Hong Kong', 'Singapore', 'Thailand', 'Israel', 'Lithuania',
       'West Indies', 'Lebanon', 'South Korea', 'Brazil', 'Canada',
       'Iceland', 'Saudi Arabia', 'Czech Republic', 'European Community'],
      dtype=object)

In [96]:
#handling outliers 

In [98]:
print(sc_df['Quantity'].describe())
print(sc_df['Price'].describe())

count    999050.000000
mean         11.433439
std         132.130328
min           1.000000
25%           1.000000
50%           4.000000
75%          12.000000
max       80995.000000
Name: Quantity, dtype: float64
count    999050.000000
mean          4.086465
std          49.482442
min           0.000000
25%           1.250000
50%           2.100000
75%           4.150000
max       25111.090000
Name: Price, dtype: float64


In [213]:
outlrs_qte = sc_df[sc_df['Quantity'] > 50]  
outlrs_qte = outlrs_qte.sort_values(by=['Quantity'])
outlrs_qte

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
812898,562104,20723,STRAWBERRY CHARLOTTE BAG,51,2011-08-02 14:31:00,2.46,Unknown,United Kingdom
769113,558475,84536A,ENGLISH ROSE NOTEBOOK A7 SIZE,51,2011-06-29 15:58:00,0.42,Unknown,United Kingdom
282336,516892,15036,ASSORTED COLOURS SILK FAN,51,2010-07-23 15:09:00,1.28,Unknown,United Kingdom
269746,515479,22297,HEART IVORY TRELLIS SMALL,51,2010-07-13 09:58:00,2.51,Unknown,United Kingdom
795888,560567,20719,WOODLAND CHARLOTTE BAG,51,2011-07-19 13:57:00,2.46,Unknown,United Kingdom
...,...,...,...,...,...,...,...,...
127166,501534,21099,SET/6 STRAWBERRY PAPER CUPS,12960,2010-03-17 13:09:00,0.10,13902,Denmark
127168,501534,21091,SET/6 WOODLAND PAPER PLATES,12960,2010-03-17 13:09:00,0.10,13902,Denmark
90857,497946,37410,BLACK AND WHITE PAISLEY FLOWER MUG,19152,2010-02-15 11:57:00,0.10,13902,Denmark
587080,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom


In [100]:
# Split data into retail and bulk sales based on a quantity threshold
bulk_threshold = 50 

scr_df = sc_df[sc_df['Quantity'] <= bulk_threshold]
bulk_df = sc_df[sc_df['Quantity'] > bulk_threshold]

In [102]:
outlrs_price = scr_df[scr_df['Price'] > 40]
outlrs_price = outlrs_price.sort_values(by=['Price'])
outlrs_price

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
337481,522290,DOT,DOTCOM POSTAGE,1,2010-09-13 14:12:00,40.04,Unknown,United Kingdom
302350,518819,DOT,DOTCOM POSTAGE,1,2010-08-12 10:36:00,40.09,Unknown,United Kingdom
359136,524195,DOT,DOTCOM POSTAGE,1,2010-09-28 09:35:00,40.13,Unknown,United Kingdom
96656,498498,DOT,DOTCOM POSTAGE,1,2010-02-19 11:07:00,40.26,Unknown,United Kingdom
981169,575636,M,Manual,1,2011-11-10 13:46:00,40.46,12473,Germany
...,...,...,...,...,...,...,...,...
342147,522796,M,Manual,1,2010-09-16 15:12:00,10468.80,Unknown,United Kingdom
135015,502265,M,Manual,1,2010-03-23 15:28:00,10953.50,Unknown,United Kingdom
135013,502263,M,Manual,1,2010-03-23 15:22:00,10953.50,12918,United Kingdom
517955,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,Unknown,United Kingdom


In [104]:
# Dropping none transactional 'StockCode'
valid_stockcode_pattern = r'(^\d{5}[A-Za-z]{0,2}$)|(^[A-Za-z]{2,4}\d{4}[A-Z]?$)|(^[B-Z]{8,9}$)'

valid_stockcode = scr_df['StockCode'].str.match(valid_stockcode_pattern)
valid_rows = scr_df[valid_stockcode]
invalid_rows = scr_df[~valid_stockcode]

In [106]:
invalid_rows['StockCode'].unique()

array(['POST', 'DOT', 'M', 'C2', 'BANK CHARGES', 'TEST001',
       'gift_0001_80', 'gift_0001_20', 'TEST002', 'gift_0001_10',
       'gift_0001_50', 'gift_0001_30', 'PADS', 'ADJUST', 'gift_0001_40',
       'gift_0001_70', 'm', 'D', 'S', 'ADJUST2', 'AMAZONFEE'],
      dtype=object)

In [108]:
invalid_rows

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.00,12682,France
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.00,12636,United States
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.00,12362,Belgium
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.00,12533,Germany
1244,489557,POST,POSTAGE,4,2009-12-01 12:52:00,18.00,12490,France
...,...,...,...,...,...,...,...,...
1067001,581498,DOT,DOTCOM POSTAGE,1,2011-12-09 10:26:00,1714.17,Unknown,United Kingdom
1067191,581570,POST,POSTAGE,1,2011-12-09 11:59:00,18.00,12662,Germany
1067228,581574,POST,POSTAGE,2,2011-12-09 12:09:00,18.00,12526,Germany
1067229,581578,POST,POSTAGE,3,2011-12-09 12:16:00,18.00,12713,Germany


In [110]:
scrv_df = valid_rows
scrv_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
...,...,...,...,...,...,...,...,...
1067365,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [114]:
# Verifying the existence of outliers after filtering none transactional data
outlrs_price_check = scrv_df[scrv_df['Price'] > 40]
outlrs_price_check = outlrs_price_check.sort_values(by=['Price'])
outlrs_price_check

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
741053,555722,21277,FRENCH STYLE EMBOSSED HEART CABINET,1,2011-06-06 16:06:00,41.63,Unknown,United Kingdom
941523,572550,23462,ROCOCO WALL MIRROR WHITE,2,2011-10-24 17:06:00,41.63,Unknown,United Kingdom
942153,572553,23462,ROCOCO WALL MIRROR WHITE,1,2011-10-24 17:09:00,41.63,Unknown,United Kingdom
863920,566557,23064,CINDERELLA CHANDELIER,4,2011-09-13 11:47:00,41.75,14646,Netherlands
759944,557527,23064,CINDERELLA CHANDELIER,4,2011-06-21 09:44:00,41.75,13941,United Kingdom
...,...,...,...,...,...,...,...,...
355259,523946,22655,VINTAGE RED KITCHEN CABINET,1,2010-09-26 11:23:00,295.00,16833,United Kingdom
268525,515349,22656,VINTAGE BLUE KITCHEN CABINET,1,2010-07-12 10:43:00,295.00,15513,United Kingdom
507927,536835,22655,VINTAGE RED KITCHEN CABINET,1,2010-12-02 18:06:00,295.00,13145,United Kingdom
748143,556446,22502,PICNIC BASKET WICKER 60 PIECES,1,2011-06-10 15:33:00,649.50,15098,United Kingdom


In [116]:
scrv_df[scrv_df['Price'] == 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
43639,493180,22003,VINTAGE BLUE VACUUM FLASK 0.5L,10,2009-12-22 11:53:00,0.0,Unknown,United Kingdom
43837,493235,79340W,WHITE ORCHID FLOWER LIGHTS,3,2009-12-22 13:45:00,0.0,Unknown,United Kingdom
43853,493251,47567,ENGLISH ROSE DESIGN KITCHEN APRON,1,2009-12-22 13:50:00,0.0,Unknown,United Kingdom


In [118]:
print(scrv_df['Quantity'].describe())
print(scrv_df['Price'].describe())

count    970147.000000
mean          7.009201
std           8.749830
min           1.000000
25%           1.000000
50%           3.000000
75%          10.000000
max          50.000000
Name: Quantity, dtype: float64
count    970147.000000
mean          3.399418
std           4.611154
min           0.000000
25%           1.250000
50%           2.100000
75%           4.150000
max         867.790000
Name: Price, dtype: float64


In [120]:
#cheking dates

In [122]:
print(scrv_df['InvoiceDate'].min(), scrv_df['InvoiceDate'].max())

2009-12-01 07:45:00 2011-12-09 12:50:00


In [124]:
#cheking null values 

In [126]:
print(scrv_df.isnull().sum())

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64


In [128]:
#now cleaning of the cancellation data set

In [130]:
c_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [132]:
c_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19494 entries, 178 to 1067178
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      19494 non-null  object        
 1   StockCode    19494 non-null  object        
 2   Description  19494 non-null  object        
 3   Quantity     19494 non-null  int64         
 4   InvoiceDate  19494 non-null  datetime64[ns]
 5   Price        19494 non-null  float64       
 6   Customer ID  19494 non-null  object        
 7   Country      19494 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 1.3+ MB


In [134]:
#handling missing values 

In [136]:
lookup_dict = df[df['Price'].ne(0)].set_index('StockCode')['Price'].to_dict()

c_df['Price'] = c_df.apply(lambda row: fill_missing_price(row, lookup_dict), axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_2832\1394168683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_df['Price'] = c_df.apply(lambda row: fill_missing_price(row, lookup_dict), axis=1)


In [138]:
c_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19494 entries, 178 to 1067178
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      19494 non-null  object        
 1   StockCode    19494 non-null  object        
 2   Description  19494 non-null  object        
 3   Quantity     19494 non-null  int64         
 4   InvoiceDate  19494 non-null  datetime64[ns]
 5   Price        19494 non-null  float64       
 6   Customer ID  19494 non-null  object        
 7   Country      19494 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 1.3+ MB


In [140]:
#fixing the quantity for analysis

In [142]:
c_df['Quantity'] = c_df['Quantity'].abs()

C:\Users\user\AppData\Local\Temp\ipykernel_2832\1465152884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_df['Quantity'] = c_df['Quantity'].abs()


In [160]:
#cleaning duplicates in the same way as retail transactions

In [146]:
c_df['RowIndex'] = c_df.index
c_df

C:\Users\user\AppData\Local\Temp\ipykernel_2832\3027773477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_df['RowIndex'] = c_df.index


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
178,C489449,22087,PAPER BUNTING WHITE LACE,12,2009-12-01 10:33:00,2.95,16321,Australia,178
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,6,2009-12-01 10:33:00,1.65,16321,Australia,179
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,4,2009-12-01 10:33:00,4.25,16321,Australia,180
181,C489449,21896,POTTING SHED TWINE,6,2009-12-01 10:33:00,2.10,16321,Australia,181
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,12,2009-12-01 10:33:00,2.95,16321,Australia,182
...,...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,11,2011-12-09 09:57:00,0.83,14397,United Kingdom,1065910
1067002,C581499,M,Manual,1,2011-12-09 10:28:00,224.69,15498,United Kingdom,1067002
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,5,2011-12-09 11:57:00,10.95,15311,United Kingdom,1067176
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,1,2011-12-09 11:58:00,1.25,17315,United Kingdom,1067177


In [148]:
dup = c_df[c_df.duplicated(subset=['Invoice', 'StockCode'], keep=False)]
dup_sort = dup.sort_values(by=['Invoice', 'StockCode'])
dup_sort

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
735,C489535,D,Discount,1,2009-12-01 12:11:00,9.00,15299,United Kingdom,735
736,C489535,D,Discount,1,2009-12-01 12:11:00,19.00,15299,United Kingdom,736
1354,C489563,90090,PINK CRYSTAL HEART PHONE CHARM,24,2009-12-01 13:09:00,0.85,13526,United Kingdom,1354
1361,C489563,90090,PINK CRYSTAL HEART PHONE CHARM,12,2009-12-01 13:09:00,0.85,13526,United Kingdom,1361
1358,C489563,90092,BLUE CRYSTAL BOOT PHONE CHARM,24,2009-12-01 13:09:00,0.85,13526,United Kingdom,1358
...,...,...,...,...,...,...,...,...,...
1045595,C580263,M,Manual,16,2011-12-02 12:43:00,0.29,12536,France,1045595
1055441,C580764,22667,RECIPE BOX RETROSPOT,12,2011-12-06 10:38:00,2.95,14562,United Kingdom,1055441
1055442,C580764,22667,RECIPE BOX RETROSPOT,12,2011-12-06 10:38:00,2.95,14562,United Kingdom,1055442
1057260,C580968,23534,WALL ART STOP FOR TEA,4,2011-12-06 14:57:00,5.95,15622,United Kingdom,1057260


In [150]:
dup_sort = dup_sort.drop_duplicates(subset=['Invoice', 'StockCode','Quantity', 'InvoiceDate'], keep='first')
dup_sort

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
735,C489535,D,Discount,1,2009-12-01 12:11:00,9.00,15299,United Kingdom,735
1354,C489563,90090,PINK CRYSTAL HEART PHONE CHARM,24,2009-12-01 13:09:00,0.85,13526,United Kingdom,1354
1361,C489563,90090,PINK CRYSTAL HEART PHONE CHARM,12,2009-12-01 13:09:00,0.85,13526,United Kingdom,1361
1358,C489563,90092,BLUE CRYSTAL BOOT PHONE CHARM,24,2009-12-01 13:09:00,0.85,13526,United Kingdom,1358
1360,C489563,90092,BLUE CRYSTAL BOOT PHONE CHARM,3,2009-12-01 13:09:00,0.85,13526,United Kingdom,1360
...,...,...,...,...,...,...,...,...,...
1045594,C580263,M,Manual,4,2011-12-02 12:43:00,9.95,12536,France,1045594
1045595,C580263,M,Manual,16,2011-12-02 12:43:00,0.29,12536,France,1045595
1055441,C580764,22667,RECIPE BOX RETROSPOT,12,2011-12-06 10:38:00,2.95,14562,United Kingdom,1055441
1057260,C580968,23534,WALL ART STOP FOR TEA,4,2011-12-06 14:57:00,5.95,15622,United Kingdom,1057260


In [152]:
dup_sort = (
    dup_sort.groupby(['Invoice', 'StockCode','InvoiceDate'], as_index=False) 
       .apply(lambda group: group.loc[group['Quantity'].idxmax()])  
       .reset_index(drop=True)
)
dup_sort

C:\Users\user\AppData\Local\Temp\ipykernel_2832\2998036414.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.loc[group['Quantity'].idxmax()])


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
0,C489535,D,Discount,1,2009-12-01 12:11:00,9.00,15299,United Kingdom,735
1,C489563,90090,PINK CRYSTAL HEART PHONE CHARM,24,2009-12-01 13:09:00,0.85,13526,United Kingdom,1354
2,C489563,90092,BLUE CRYSTAL BOOT PHONE CHARM,24,2009-12-01 13:09:00,0.85,13526,United Kingdom,1358
3,C489632,20814,SILVER FINCH DECORATION,12,2009-12-01 15:18:00,1.25,13381,United Kingdom,2866
4,C490133,21314,SMALL GLASS HEART TRINKET POT,504,2009-12-03 18:36:00,1.85,17949,United Kingdom,9324
...,...,...,...,...,...,...,...,...,...
485,C577330,S,SAMPLES,1,2011-11-18 14:16:00,2.89,Unknown,United Kingdom,1006422
486,C578073,M,Manual,36,2011-11-22 16:02:00,0.32,18139,United Kingdom,1017116
487,C580263,M,Manual,16,2011-12-02 12:43:00,0.29,12536,France,1045595
488,C580764,22667,RECIPE BOX RETROSPOT,12,2011-12-06 10:38:00,2.95,14562,United Kingdom,1055441


In [154]:
dup_rows = dup[~dup['RowIndex'].isin(dup_sort['RowIndex'])]
dup_rows

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
736,C489535,D,Discount,1,2009-12-01 12:11:00,19.00,15299,United Kingdom,736
1360,C489563,90092,BLUE CRYSTAL BOOT PHONE CHARM,3,2009-12-01 13:09:00,0.85,13526,United Kingdom,1360
1361,C489563,90090,PINK CRYSTAL HEART PHONE CHARM,12,2009-12-01 13:09:00,0.85,13526,United Kingdom,1361
2867,C489632,20814,SILVER FINCH DECORATION,1,2009-12-01 15:18:00,1.25,13381,United Kingdom,2867
9325,C490133,21314,SMALL GLASS HEART TRINKET POT,252,2009-12-03 18:36:00,1.85,17949,United Kingdom,9325
...,...,...,...,...,...,...,...,...,...
1006424,C577330,S,SAMPLES,1,2011-11-18 14:16:00,5.99,Unknown,United Kingdom,1006424
1017117,C578073,M,Manual,36,2011-11-22 16:02:00,0.56,18139,United Kingdom,1017117
1045594,C580263,M,Manual,4,2011-12-02 12:43:00,9.95,12536,France,1045594
1055442,C580764,22667,RECIPE BOX RETROSPOT,12,2011-12-06 10:38:00,2.95,14562,United Kingdom,1055442


In [156]:
cc_df = c_df[~c_df['RowIndex'].isin(dup_rows['RowIndex'])]
cc_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,RowIndex
178,C489449,22087,PAPER BUNTING WHITE LACE,12,2009-12-01 10:33:00,2.95,16321,Australia,178
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,6,2009-12-01 10:33:00,1.65,16321,Australia,179
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,4,2009-12-01 10:33:00,4.25,16321,Australia,180
181,C489449,21896,POTTING SHED TWINE,6,2009-12-01 10:33:00,2.10,16321,Australia,181
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,12,2009-12-01 10:33:00,2.95,16321,Australia,182
...,...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,11,2011-12-09 09:57:00,0.83,14397,United Kingdom,1065910
1067002,C581499,M,Manual,1,2011-12-09 10:28:00,224.69,15498,United Kingdom,1067002
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,5,2011-12-09 11:57:00,10.95,15311,United Kingdom,1067176
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,1,2011-12-09 11:58:00,1.25,17315,United Kingdom,1067177


In [158]:
cc_df = cc_df.drop(columns=['RowIndex'])

In [162]:
#checking consitancy 

In [164]:
cc_df['Country'] = cc_df['Country'].replace(country_mapping)
cc_df['Country'].unique()

array(['Australia', 'United Kingdom', 'France', 'Japan', 'Germany',
       'Nigeria', 'Spain', 'Italy', 'Ireland', 'Denmark', 'Sweden',
       'Channel Islands', 'United Arab Emirates', 'Cyprus', 'Belgium',
       'Finland', 'Netherlands', 'Portugal', 'Unspecified', 'Malta',
       'Norway', 'Greece', 'Bahrain', 'Switzerland', 'Austria',
       'Hong Kong', 'Poland', 'United States', 'South Korea', 'Lebanon',
       'South Africa', 'Israel', 'Saudi Arabia', 'Czech Republic',
       'Singapore', 'European Community'], dtype=object)

In [176]:
#handling outliers the same way as in the retail analysis

In [178]:
print(cc_df['Quantity'].describe())
print(cc_df['Price'].describe())

count    18910.000000
mean        25.099524
std        814.532578
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max      80995.000000
Name: Quantity, dtype: float64
count    18910.000000
mean        42.632274
std        577.179845
min          0.010000
25%          1.450000
50%          2.950000
75%          6.350000
max      38970.000000
Name: Price, dtype: float64


In [180]:
outlrs_qte = cc_df[cc_df['Quantity'] > 50]  
outlrs_qte = outlrs_qte.sort_values(by=['Quantity'])
outlrs_qte

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
501428,C536164,84988,SET OF 72 PINK HEART PAPER DOILIES,51,2010-11-30 12:09:00,1.42,15369,United Kingdom
702233,C552021,22963,JAM JAR WITH GREEN LID,51,2011-05-05 17:57:00,0.72,15125,United Kingdom
198053,C508458,79323P,PINK CHERRY LIGHTS,51,2010-05-14 17:15:00,5.45,15270,United Kingdom
198054,C508458,79323W,WHITE CHERRY LIGHTS,51,2010-05-14 17:15:00,5.45,15270,United Kingdom
294487,C518066,21934,SKULL SHOULDER BAG,51,2010-08-04 12:45:00,1.65,17841,United Kingdom
...,...,...,...,...,...,...,...,...
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,7008,2010-09-28 11:02:00,0.13,14277,France
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,7128,2010-09-28 11:02:00,0.08,14277,France
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,9360,2010-12-02 14:23:00,0.03,15838,United Kingdom
587085,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:17:00,1.04,12346,United Kingdom


In [174]:
ccr_df = cc_df[cc_df['Quantity'] <= bulk_threshold]
bulkc_df = cc_df[cc_df['Quantity'] > bulk_threshold]

In [182]:
# Split data into retail and bulk sales based on a quantity threshold
outlrs_price = ccr_df[ccr_df['Price'] > 40]
outlrs_price = outlrs_price.sort_values(by=['Price'])
outlrs_price

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
981170,C575638,M,Manual,1,2011-11-10 13:48:00,40.46,12473,Germany
118495,C500700,M,Manual,1,2010-03-09 13:20:00,40.68,Unknown,United Kingdom
87050,C497505,D,Discount,1,2010-02-10 11:25:00,41.12,14593,United Kingdom
750895,C556678,23064,CINDERELLA CHANDELIER,4,2011-06-13 17:21:00,41.75,13658,United Kingdom
927904,C571499,23064,CINDERELLA CHANDELIER,10,2011-10-17 15:07:00,41.75,12454,Spain
...,...,...,...,...,...,...,...,...
569163,C540117,AMAZONFEE,AMAZON FEE,1,2011-01-05 09:55:00,16888.02,Unknown,United Kingdom
1050063,C580605,AMAZONFEE,AMAZON FEE,1,2011-12-05 11:36:00,17836.46,Unknown,United Kingdom
320581,C520667,BANK CHARGES,Bank Charges,1,2010-08-27 13:42:00,18910.69,Unknown,United Kingdom
241824,C512770,M,Manual,1,2010-06-17 16:52:00,25111.09,17399,United Kingdom


In [184]:
ccr_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,12,2009-12-01 10:33:00,2.95,16321,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,6,2009-12-01 10:33:00,1.65,16321,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,4,2009-12-01 10:33:00,4.25,16321,Australia
181,C489449,21896,POTTING SHED TWINE,6,2009-12-01 10:33:00,2.10,16321,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,12,2009-12-01 10:33:00,2.95,16321,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,11,2011-12-09 09:57:00,0.83,14397,United Kingdom
1067002,C581499,M,Manual,1,2011-12-09 10:28:00,224.69,15498,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,5,2011-12-09 11:57:00,10.95,15311,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [186]:
ccr_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18255 entries, 178 to 1067178
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      18255 non-null  object        
 1   StockCode    18255 non-null  object        
 2   Description  18255 non-null  object        
 3   Quantity     18255 non-null  int64         
 4   InvoiceDate  18255 non-null  datetime64[ns]
 5   Price        18255 non-null  float64       
 6   Customer ID  18255 non-null  object        
 7   Country      18255 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 1.3+ MB


In [188]:
valid_stockcode = ccr_df['StockCode'].str.match(valid_stockcode_pattern).fillna(True)
valid_rows = ccr_df[valid_stockcode]
invalid_rows = ccr_df[~valid_stockcode]

C:\Users\user\AppData\Local\Temp\ipykernel_2832\2350890272.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valid_stockcode = ccr_df['StockCode'].str.match(valid_stockcode_pattern).fillna(True)


In [190]:
invalid_rows['StockCode'].unique()

array(['D', 'POST', 'M', 'BANK CHARGES', 'gift_0001_80', 'ADJUST', 'S',
       'C2', 'PADS', 'DOT', 'TEST001', 'AMAZONFEE', 'CRUK'], dtype=object)

In [192]:
ccrv_df = valid_rows
ccrv_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,12,2009-12-01 10:33:00,2.95,16321,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,6,2009-12-01 10:33:00,1.65,16321,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,4,2009-12-01 10:33:00,4.25,16321,Australia
181,C489449,21896,POTTING SHED TWINE,6,2009-12-01 10:33:00,2.10,16321,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,12,2009-12-01 10:33:00,2.95,16321,Australia
...,...,...,...,...,...,...,...,...
1065909,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,12,2011-12-09 09:57:00,1.95,14397,United Kingdom
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,11,2011-12-09 09:57:00,0.83,14397,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,5,2011-12-09 11:57:00,10.95,15311,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [194]:
print(ccrv_df['Quantity'].describe())
print(ccrv_df['Price'].describe())

count    17219.000000
mean         5.177827
std          7.975531
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max         50.000000
Name: Quantity, dtype: float64
count    17219.000000
mean         4.699713
std          9.454229
min          0.060000
25%          1.490000
50%          2.950000
75%          5.950000
max        295.000000
Name: Price, dtype: float64


In [198]:
#now cleaning bulk data in the same way

In [200]:
bulk_df.tail(60)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1063244,581338,23344,JUMBO BAG 50'S CHRISTMAS,140,2011-12-08 12:12:00,1.79,14646,Netherlands
1063344,581375,21137,BLACK RECORD COVER FRAME,960,2011-12-08 12:36:00,3.39,16210,United Kingdom
1063526,581385,21479,WHITE SKULL HOT WATER BOTTLE,72,2011-12-08 13:11:00,3.75,13298,United Kingdom
1063965,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,1.55,Unknown,United Kingdom
1063966,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,1.45,Unknown,United Kingdom
1064029,581410,84949,SILVER HANGING T-LIGHT HOLDER,72,2011-12-08 14:14:00,1.45,13098,United Kingdom
1064034,581410,23582,VINTAGE DOILY JUMBO BAG RED,100,2011-12-08 14:14:00,1.79,13098,United Kingdom
1064038,581410,23545,WRAP RED VINTAGE DOILY,75,2011-12-08 14:14:00,0.42,13098,United Kingdom
1064045,581410,23084,RABBIT NIGHT LIGHT,168,2011-12-08 14:14:00,1.79,13098,United Kingdom
1064049,581410,22086,PAPER CHAIN KIT 50'S CHRISTMAS,80,2011-12-08 14:14:00,2.55,13098,United Kingdom


In [202]:
valid_stockcode_pattern = r'(^\d{5}[A-Za-z]{0,2}$)|(^[A-Za-z]{2,4}\d{4}[A-Z]?$)|(^[B-Z]{8,9}$)'

In [204]:
valid_stockcode = bulk_df['StockCode'].str.match(valid_stockcode_pattern).fillna(True)
valid_rows = bulk_df[valid_stockcode]
invalid_rows = bulk_df[~valid_stockcode]

In [206]:
invalid_rows['StockCode'].unique()

array(['C2', 'M', 'POST', 'DOT', 'D'], dtype=object)

In [208]:
bulkv_df = valid_rows
bulkv_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
58,489438,20711,JUMBO BAG TOYS,60,2009-12-01 09:24:00,1.30,18102,United Kingdom
61,489438,84031A,CHARLIE+LOLA RED HOT WATER BOTTLE,56,2009-12-01 09:24:00,3.00,18102,United Kingdom
62,489438,84031B,CHARLIE LOLA BLUE HOT WATER BOTTLE,56,2009-12-01 09:24:00,3.00,18102,United Kingdom
63,489438,84032A,CHARLIE+LOLA PINK HOT WATER BOTTLE,60,2009-12-01 09:24:00,1.90,18102,United Kingdom
64,489438,84032B,CHARLIE + LOLA RED HOT WATER BOTTLE,56,2009-12-01 09:24:00,1.90,18102,United Kingdom
...,...,...,...,...,...,...,...,...
1067163,581566,23404,HOME SWEET HOME BLACKBOARD,144,2011-12-09 11:50:00,3.26,18102,United Kingdom
1067164,581567,21417,COCKLE SHELL DISH,84,2011-12-09 11:56:00,0.79,16626,United Kingdom
1067172,581567,21326,AGED GLASS SILVER T-LIGHT HOLDER,144,2011-12-09 11:56:00,0.55,16626,United Kingdom
1067207,581571,23167,SMALL CERAMIC TOP STORAGE JAR,96,2011-12-09 12:00:00,0.69,15311,United Kingdom


In [210]:
bulkc_df.sample(60)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
797551,C560733,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,360,2011-07-20 14:55:00,2.75,15298,United Kingdom
350018,C523402,84568,GIRLS ALPHABET IRON ON PATCHES,288,2010-09-21 20:24:00,0.21,15291,United Kingdom
407751,C528406,21098,CHRISTMAS TOILET ROLL,72,2010-10-22 09:09:00,1.06,17829,United Arab Emirates
765686,C558095,23210,WHITE ROCKING HORSE HAND PAINTED,144,2011-06-26 13:13:00,1.04,15046,United Kingdom
17674,C490807,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,96,2009-12-08 12:25:00,0.21,13091,United Kingdom
559160,C539287,85172,HYACINTH BULB T-LIGHT CANDLES,80,2010-12-16 16:24:00,0.42,17655,United Kingdom
3541,C489713,21094,SET/6 RED SPOTTY PAPER PLATES,150,2009-12-02 10:55:00,0.64,12755,Japan
685604,C550456,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,2011-04-18 13:08:00,2.55,15749,United Kingdom
749880,C556522,22920,HERB MARKER BASIL,1515,2011-06-13 11:21:00,0.55,16938,United Kingdom
233095,C511965,16033,MINI HIGHLIGHTER PENS,120,2010-06-11 13:34:00,0.12,14727,United Kingdom


In [212]:
valid_stockcode = bulkc_df['StockCode'].str.match(valid_stockcode_pattern).fillna(True)
valid_rows = bulkc_df[valid_stockcode]
invalid_rows = bulkc_df[~valid_stockcode]

C:\Users\user\AppData\Local\Temp\ipykernel_2832\3255478372.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valid_stockcode = bulkc_df['StockCode'].str.match(valid_stockcode_pattern).fillna(True)


In [214]:
invalid_rows['StockCode'].unique()

array(['M', 'D'], dtype=object)

In [216]:
bulkcv_df = valid_rows
bulkcv_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1357,C489563,90085,CRYSTAL STILETTO PHONE CHARM,81,2009-12-01 13:09:00,0.85,13526,United Kingdom
3541,C489713,21094,SET/6 RED SPOTTY PAPER PLATES,150,2009-12-02 10:55:00,0.64,12755,Japan
9322,C490133,72045D,ROSES ON BLUE TEACUP CANDLE,504,2009-12-03 18:36:00,1.69,17949,United Kingdom
9323,C490133,20971,PINK BLUE FELT CRAFT TRINKET BOX,600,2009-12-03 18:36:00,1.06,17949,United Kingdom
9324,C490133,21314,SMALL GLASS HEART TRINKET POT,504,2009-12-03 18:36:00,1.85,17949,United Kingdom
...,...,...,...,...,...,...,...,...
1056755,C580913,22536,MAGIC DRAWING SLATE PURDEY,120,2011-12-06 13:41:00,0.36,17857,United Kingdom
1057280,C580971,22313,OFFICE MUG WARMER PINK,250,2011-12-06 15:04:00,2.55,14298,United Kingdom
1063076,C581330,16169E,WRAP 50'S CHRISTMAS,100,2011-12-08 11:57:00,0.42,15877,United Kingdom
1065540,C581462,21642,ASSORTED TUTTI FRUTTI PEN,72,2011-12-08 18:51:00,0.29,12985,United Kingdom


In [218]:
#decently cleansed dataset

In [220]:
scrv_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
...,...,...,...,...,...,...,...,...
1067365,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [222]:
ccrv_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,12,2009-12-01 10:33:00,2.95,16321,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,6,2009-12-01 10:33:00,1.65,16321,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,4,2009-12-01 10:33:00,4.25,16321,Australia
181,C489449,21896,POTTING SHED TWINE,6,2009-12-01 10:33:00,2.10,16321,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,12,2009-12-01 10:33:00,2.95,16321,Australia
...,...,...,...,...,...,...,...,...
1065909,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,12,2011-12-09 09:57:00,1.95,14397,United Kingdom
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,11,2011-12-09 09:57:00,0.83,14397,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,5,2011-12-09 11:57:00,10.95,15311,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [224]:
bulkv_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
58,489438,20711,JUMBO BAG TOYS,60,2009-12-01 09:24:00,1.3,18102,United Kingdom
61,489438,84031A,CHARLIE+LOLA RED HOT WATER BOTTLE,56,2009-12-01 09:24:00,3.0,18102,United Kingdom
62,489438,84031B,CHARLIE LOLA BLUE HOT WATER BOTTLE,56,2009-12-01 09:24:00,3.0,18102,United Kingdom
63,489438,84032A,CHARLIE+LOLA PINK HOT WATER BOTTLE,60,2009-12-01 09:24:00,1.9,18102,United Kingdom
64,489438,84032B,CHARLIE + LOLA RED HOT WATER BOTTLE,56,2009-12-01 09:24:00,1.9,18102,United Kingdom


In [226]:
bulkcv_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1357,C489563,90085,CRYSTAL STILETTO PHONE CHARM,81,2009-12-01 13:09:00,0.85,13526,United Kingdom
3541,C489713,21094,SET/6 RED SPOTTY PAPER PLATES,150,2009-12-02 10:55:00,0.64,12755,Japan
9322,C490133,72045D,ROSES ON BLUE TEACUP CANDLE,504,2009-12-03 18:36:00,1.69,17949,United Kingdom
9323,C490133,20971,PINK BLUE FELT CRAFT TRINKET BOX,600,2009-12-03 18:36:00,1.06,17949,United Kingdom
9324,C490133,21314,SMALL GLASS HEART TRINKET POT,504,2009-12-03 18:36:00,1.85,17949,United Kingdom


In [228]:
# Online retail II cleaned, segmented, and saved 
scrv_df.to_csv('data\\processed\\retail_cleaned.csv', index=False)

bulkv_df.to_csv('data\\processed\\bulk_sales_cleaned.csv', index=False)

ccrv_df.to_csv('data\\processed\\cancellations_cleaned.csv', index=False)

bulkcv_df.to_csv('data\\processed\\bulk_cancellations_cleaned.csv', index=False)

print("done")

done
